# Player Statistics and Attributes

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import plotly.express as px

import sqlite3

from nba_api.stats.endpoints import PlayerCareerStats

In [ ]:
# you might need to pip install the following libraries
#pip install nba_api
#pip install plotly.express
#pip install kaleido

## Get all player ID

In [2]:
#path = ".." #Everything preceding the file name
database = 'basketball.sqlite' #The path + the file name

conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
#tables

In [33]:
Player = pd.read_sql(
    """
        SELECT cast(id as int) as id, full_name, first_name, last_name, is_active from Player order by id
    """,
    conn)
Player.to_csv('Player.csv', sep=',')

#Player['id'] = Player['id'].astype('int')
Player

,id,full_name,first_name,last_name,is_active
0,2,Byron Scott,Byron,Scott,0
1,3,Grant Long,Grant,Long,0
2,7,Dan Schayes,Dan,Schayes,0
3,9,Sedale Threatt,Sedale,Threatt,0
4,12,Chris King,Chris,King,0
...,...,...,...,...,...
4496,1629744,Matt Thomas,Matt,Thomas,1
4497,1629745,Tariq Owens,Tariq,Owens,1
4498,1629750,Javonte Green,Javonte,Green,1
4499,1629752,Juwan Morgan,Juwan,Morgan,1


## Get Player attribute

In [4]:
Player_Attributes = pd.read_sql(
    """
    SELECT 
    pa.ID, p.is_active, cast(BIRTHDATE as date) as BIRTHDATE, HEIGHT, WEIGHT, JERSEY, POSITION,SCHOOL, COUNTRY, 
    TEAM_ID, GAMES_PLAYED_CURRENT_SEASON_FLAG,SEASON_EXP,
    FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,
    DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PTS,AST,REB,ALL_STAR_APPEARANCES,PIE
    FROM Player_Attributes pa
    left join Player p on p.id = pa.id
    where team_id <>0

    """, 
    conn)

Player_Attributes

,ID,is_active,BIRTHDATE,HEIGHT,WEIGHT,JERSEY,POSITION,SCHOOL,COUNTRY,TEAM_ID,...,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PTS,AST,REB,ALL_STAR_APPEARANCES,PIE
0,76001,0,1968,82.0,240.0,30,Forward,Duke,USA,1610612757,...,Y,Y,1990,1,25,5.7,0.3,3.3,0.0,NaN
1,76002,0,1946,81.0,235.0,54,Center,Iowa State,USA,1610612745,...,Y,Y,1968,1,5,9.0,1.2,8.0,0.0,NaN
2,76003,0,1947,86.0,225.0,33,Center,UCLA,USA,1610612747,...,Y,Y,1969,1,1,24.6,3.6,11.2,18.0,NaN
3,51,0,1969,73.0,162.0,1,Guard,Louisiana State,USA,1610612743,...,Y,Y,1990,1,3,14.6,3.5,1.9,0.0,NaN
4,1505,0,1974,78.0,235.0,9,Forward-Guard,San Jose State,France,1610612758,...,Y,Y,1997,1,11,7.8,1.1,3.3,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,1627790,1,1997,82.0,266.0,41,Center,,Croatia,1610612739,...,Y,Y,2016,1,23,6.0,0.6,3.9,0.0,NaN
3832,78647,0,1953,85.0,240.0,34,Center,Kent State,USA,1610612765,...,Y,Y,Undrafted,Undrafted,Undrafted,0.3,0.1,1.1,0.0,NaN
3833,78648,0,1948,73.0,170.0,6,Guard,Duquesne,USA,1610612749,...,Y,Y,1970,2,33,2.2,1.4,0.9,0.0,NaN
3834,1627826,1,1997,84.0,240.0,40,Center,Mega Basket,Croatia,1610612746,...,Y,Y,2016,2,32,8.5,1.1,7.0,NaN,0.126


## Get Player stats from nba_api
https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/playercareerstats.md

Parameter
- per_mode36: (Totals)|(PerGame)|(Per36)
- player_id: ex) 2544
- LeagueID: nullable


#options of tables:   
player_info.career_totals_all_star_season.get_data_frame()    
player_info.career_totals_college_season.get_data_frame()    
player_info.career_totals_post_season.get_data_frame()    
player_info.career_totals_regular_season.get_data_frame()    
player_info.season_rankings_post_season.get_data_frame()    
player_info.season_rankings_regular_season.get_data_frame()    
player_info.season_totals_all_star_season.get_data_frame()    
player_info.season_totals_college_season.get_data_frame()    
player_info.season_totals_regular_season.get_data_frame()    

In [11]:
player_info_0 = PlayerCareerStats(player_id=Player['id'][0], per_mode36 = 'Per36')
player_stats_df = player_info_0.career_totals_regular_season.get_data_frame()

In [45]:
player_stats_df

,PLAYER_ID,LEAGUE_ID,Team_ID,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,3,00,0,1003,673,28518.0,4.4,9.4,0.467,0.1,...,0.761,2.7,5,7.8,2.2,1.5,0,2.1,3.7,12.0
0,7,00,0,1138,427,21976.0,4.9,10.2,0.481,0,...,0.806,3,6.3,9.3,2.1,0.9,0,2.6,5.7,14.4
0,9,00,0,951,304,22437.0,6.3,13.0,0.485,0.3,...,0.815,0.6,2.1,2.7,5.8,1.8,0,2.1,3.5,15.0
0,12,00,0,103,66,2058.0,4.7,11.2,0.423,0.8,...,0.633,1.9,3.6,5.4,2.0,1.3,0,2.1,3.2,12.1
0,15,00,0,789,250,14612.0,5.0,11.6,0.434,2.1,...,0.839,1.3,3,4.2,1.9,1,0,1.5,2.8,14.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1629744,00,0,87,1,769.0,5.8,13.0,0.442,3.5,...,0.808,0.7,4.2,4.9,1.9,0.7,0,1.4,2.6,16.0
0,1629745,00,0,3,0,15.0,2.4,12.0,0.200,0,...,1.000,4.8,2.4,7.2,0.0,2.4,0,0,2.4,9.6
0,1629750,00,0,95,5,1043.0,4.4,8.4,0.523,0.8,...,0.722,1.8,4.4,6.2,1.6,1.9,0,1.5,3.5,11.8
0,1629752,00,0,50,0,282.0,3.7,7.1,0.518,0.9,...,0.545,2.4,4.9,7.3,1.9,0.6,0,1.3,5.1,9.1


In [43]:
for i,v in enumerate(Player['id'][1:]):
    player_info = PlayerCareerStats(player_id=v, per_mode36 = 'Per36')
    player_info_df = player_info.career_totals_regular_season.get_data_frame()
    player_stats_df = player_stats_df.append(player_info_df)
    print(v)   
    

78116
78117
78118
78120
78121
78123
78124
78125
78126
78128
78130
78131
78132
78133
78135
78136
78138
78139
78140
78141
78142
78143
78144
78145
78146
78147
78148
78149
78150
78151
78152
78153
78154
78157
78159
78160
78161
78162
78163
78164
78167
78168
78169
78170
78171
78172
78173
78174
78175
78176
78177
78178
78179
78181
78182
78183
78184
78185
78187
78188
78189
78190
78191
78192
78194
78195
78197
78198
78199
78200
78201
78202
78203
78204
78205
78207
78208
78209
78212
78213
78214
78215
78216
78217
78219
78220
78221
78222
78223
78225
78226
78227
78230
78231
78232
78233
78234
78235
78237
78238
78239
78240
78241
78243
78244
78245
78247
78249
78250
78251
78252
78253
78256
78257
78259
78260
78261
78264
78266
78267
78268
78269
78270
78271
78273
78274
78275
78276
78277
78278
78279
78280
78281
78283
78284
78285
78286
78287
78291
78293
78294
78295
78296
78297
78298
78299
78301
78302
78303
78304
78305
78306
78307
78308
78309
78310
78311
78312
78313
78314
78316
78317
78318
78320
78321
78322
7832

1626161
1626162
1626163
1626164
1626166
1626167
1626168
1626169
1626170
1626171
1626172
1626173
1626174
1626175
1626176
1626177
1626178
1626179
1626181
1626183
1626184
1626185
1626187
1626188
1626191
1626192
1626195
1626196
1626199
1626202
1626203
1626204
1626205
1626209
1626210
1626214
1626220
1626224
1626242
1626245
1626246
1626251
1626253
1626254
1626257
1626259
1626262
1626266
1626273
1626296
1626643
1626780
1627098
1627215
1627293
1627362
1627395
1627732
1627733
1627734
1627735
1627736
1627737
1627738
1627739
1627740
1627741
1627742
1627743
1627744
1627745
1627746
1627747
1627748
1627749
1627750
1627751
1627752
1627753
1627754
1627755
1627756
1627757
1627758
1627759
1627761
1627762
1627763
1627767
1627770
1627771
1627772
1627773
1627774
1627775
1627777
1627778
1627779
1627780
1627781
1627782
1627783
1627784
1627785
1627786
1627787
1627788
1627789
1627790
1627791
1627812
1627814
1627815
1627816
1627817
1627819
1627820
1627821
1627823
1627824
1627826
1627827
1627832
1627834
1627835


In [44]:
player_stats_df = player_stats_df.drop_duplicates(subset=['PLAYER_ID'], keep=False)
